In [ ]:
from typing import Any

DEEPSEEK_API_KEY="your api_key"
BASE_URL="https://api.deepseek.com"

工具是Agent实现智能的重大入口，除了日常使用的第三方提供的Tools之外，开发者必须学会自定义自己的Tools.  
Tool的三大关键参数：  
* 1 name: str
* 2 description: str
* 3 args_schema: Pydantic BaseModel
  
自定义Tool有三种形式：
* 1 自定义函数：使用@tool装饰器
* 2 自定义类：继承BaseTool类实现_run方法和异步的_arun方法
* 3 Structured Tool.from_function()
### 1 自定义函数

In [3]:
from langchain.tools import tool
from langchain_core.pydantic_v1 import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="需要查询的句子")
    
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """在线查找内容"""
    return "Welcome to LangChain!!"

print(search.name)
print(search.description)
print(search.args)

search
在线查找内容
{'query': {'title': 'Query', 'description': '需要查询的句子', 'type': 'string'}}


### 2 继承BaseTool

In [6]:
from langchain.tools import BaseTool
from typing import Type
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.callbacks.manager import CallbackManagerForToolRun, AsyncCallbackManagerForToolRun

class SearchInput(BaseModel):
    query: str = Field(description="需要查询的句子")

class SearchTool(BaseTool):
    name: str = "search"
    description: str = "在线查找内容"
    args_schema: Type[BaseModel] = SearchInput
    
    def _run(self, query: str, run_manager: CallbackManagerForToolRun = None)-> str:
        return "Welcome to LangChain!"
    
    def _arun(self, query: str, run_manager: AsyncCallbackManagerForToolRun = None) -> str:
        raise NotImplementedError("不支持异步")
    
search = SearchTool()
print(search.name)
print(search.description)
print(search.args)

search
在线查找内容
{'query': {'title': 'Query', 'description': '需要查询的句子', 'type': 'string'}}


### 3 StructuredTool类

In [8]:
from langchain.tools import tool, StructuredTool
from langchain_core.pydantic_v1 import BaseModel, Field

class SearchInput(BaseModel):
    query: str = Field(description="需要查询的句子")
    
def search(query: str) -> str:
    """在线查找内容"""
    return "Welcome to LangChain!!"

search = StructuredTool.from_function(
    func=search, description="在线查找内容", args_schema=SearchInput
)
print(search.name)
print(search.description)
print(search.args)

search
在线查找内容
{'query': {'title': 'Query', 'description': '需要查询的句子', 'type': 'string'}}


### 4 convert_to_openai_functions  
该方式可以用于提供tool的openai_functions的形式

In [9]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = convert_to_openai_function(search)
print(functions)

{'name': 'search', 'description': '在线查找内容', 'parameters': {'type': 'object', 'properties': {'query': {'description': '需要查询的句子', 'type': 'string'}}, 'required': ['query']}}
